In [19]:
import json
import numpy as np

In [130]:
class Card:
    
    def __init__(self, numbers, name=None, joker=0):
        x = np.array(numbers)
        assert x.size == 24
        assert np.all(x[:-1] <= x[1:])
        assert all([np.issubdtype(k, np.integer) for k in x])
        self._numbers = tuple(x)
        self._name = name
        self._joker = joker
        self._matrix = None
    
    def __str__(self):
        return "<Card:'{}' {}>".format(self._name, self.numbers)
    
    def __repr__(self):
        return str(self)
    
    @property
    def numbers(self):
        return self._numbers
    
    @property
    def values(self):
        return np.array(self.numbers)
    
    def spacer(self, numbers):
        return list(numbers[:12]) + [self._joker] + list(numbers[12:])

    @property
    def matrix(self):
        if self._matrix is None:
            self._matrix = np.array(self.spacer(self.numbers)).reshape((5,5)).T
        return self._matrix
    
    def rows(self, tagged=True, creator=tuple):
        if tagged:
            return {"{}.R{}".format(self._name[0], k): creator(self.matrix[k, :]) for k in [0,1,3,4]}
        else:
            return tuple([creator(self.matrix[k, :]) for k in [0,1,3,4]])
    
    def columns(self, tagged=True, creator=tuple):
        if tagged:
            return {"{}.C{}".format(self._name[0], k): creator(self.matrix[:, k]) for k in [0,1,3,4]}
        else:
            return tuple([creator(self.matrix[k, :]) for k in [0,1,3,4]])
        
    def specials(self, tagged=True, creator=tuple):
        labels = ["D1S", "D2S", "R2S", "C2S"]
        sols = [
            np.diag(self.matrix),
            np.array(list(reversed(np.diag(np.fliplr(self.matrix))))),
            self.matrix[2,:],
            self.matrix[:,2],
        ]
        if tagged:
            return {"{}.{}".format(self._name[0], k): creator(sols[i]) for i,k in enumerate(labels) }
        else:
            return tuple([creator(sol) for sol in sols])

    def solutions(self, tagged=True, creator=tuple):
        if tagged:
            sols = dict()
            sols.update(self.rows(creator=creator))
            sols.update(self.columns(creator=creator))
            sols.update(self.specials(creator=creator))
            return sols
        else:
            sols = []
            sols.extend(self.rows(tagged=False, creator=creator))
            sols.extend(self.columns(tagged=False, creator=creator))
            sols.extend(self.specials(tagged=False, creator=creator))
            return tuple(sols)
        

In [131]:
with open('SGRS2020_Task25_data.json') as fh:
    data = json.load(fh)

In [132]:
Cards = {k[0]: Card(v, name=k) for (k, v) in data.items()}
Cards

{'A': <Card:'Alice' (2, 3, 10, 12, 15, 19, 23, 24, 25, 30, 34, 37, 41, 43, 47, 48, 53, 56, 57, 61, 64, 68, 72, 73)>,
 'B': <Card:'Bob' (1, 2, 6, 12, 15, 17, 18, 21, 23, 24, 33, 37, 39, 40, 48, 49, 50, 52, 53, 63, 65, 66, 67, 68)>,
 'C': <Card:'Colby' (3, 4, 7, 9, 10, 19, 23, 25, 28, 29, 34, 35, 42, 45, 47, 48, 49, 50, 53, 61, 63, 67, 71, 75)>,
 'D': <Card:'Dylan' (1, 3, 6, 8, 12, 22, 26, 27, 28, 29, 34, 36, 41, 42, 54, 55, 56, 57, 58, 68, 70, 71, 74, 75)>,
 'E': <Card:'Emily' (1, 2, 3, 8, 12, 22, 23, 24, 27, 29, 35, 41, 44, 45, 51, 52, 53, 55, 58, 61, 66, 72, 74, 75)>,
 'F': <Card:'Faith' (4, 5, 11, 13, 14, 17, 25, 27, 28, 30, 33, 36, 38, 41, 48, 49, 56, 57, 60, 62, 63, 65, 66, 67)>,
 'G': <Card:'Grace' (1, 3, 6, 10, 12, 25, 26, 27, 29, 30, 32, 38, 43, 44, 48, 55, 56, 58, 59, 64, 68, 69, 73, 75)>,
 'H': <Card:'Harris' (1, 4, 5, 7, 8, 19, 26, 27, 28, 29, 34, 35, 42, 45, 54, 55, 56, 57, 58, 61, 62, 67, 70, 73)>,
 'I': <Card:'Isla' (2, 3, 8, 12, 15, 17, 18, 21, 23, 24, 33, 36, 43, 45, 48,

In [133]:
Cards['A'].matrix

array([[ 2, 19, 34, 47, 61],
       [ 3, 23, 37, 48, 64],
       [10, 24,  0, 53, 68],
       [12, 25, 41, 56, 72],
       [15, 30, 43, 57, 73]])

In [135]:
Cards['A'].columns(tagged=True)

{'A.C0': (2, 3, 10, 12, 15),
 'A.C1': (19, 23, 24, 25, 30),
 'A.C3': (47, 48, 53, 56, 57),
 'A.C4': (61, 64, 68, 72, 73)}

In [140]:
Cards['A'].specials(creator=np.array)

{'A.D1S': array([ 2, 23,  0, 56, 73]),
 'A.D2S': array([15, 25,  0, 48, 61]),
 'A.R2S': array([10, 24,  0, 53, 68]),
 'A.C2S': array([34, 37,  0, 41, 43])}

In [139]:
Cards['A'].solutions(tagged=False, creator=set)

({2, 19, 34, 47, 61},
 {3, 23, 37, 48, 64},
 {12, 25, 41, 56, 72},
 {15, 30, 43, 57, 73},
 {2, 19, 34, 47, 61},
 {3, 23, 37, 48, 64},
 {12, 25, 41, 56, 72},
 {15, 30, 43, 57, 73},
 {0, 2, 23, 56, 73},
 {0, 15, 25, 48, 61},
 {0, 10, 24, 53, 68},
 {0, 34, 37, 41, 43})